In [1]:
import pyspark
from delta import *

In [2]:
warehouse_location = 'hdfs://hdfs:9000/hive/warehouse'

In [3]:
builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.warehouse.dir", warehouse_location)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-359dc0a9-0a30-4797-9d21-31529b4b58a5;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 667ms :: artifacts dl 30ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-runtime;3.5.2 fro

In [4]:
spark

In [5]:
path = 'hdfs://hdfs:9000/user/range-temp-table'

In [6]:
data = spark.range(0, 5)
data.write.format('delta').mode('overwrite').save(path)

In [7]:
df = spark.read.format('delta').load(path)
df.show()

+---+
| id|
+---+
|  2|
|  3|
|  4|
|  0|
|  1|
+---+



In [8]:
data = spark.range(5, 10)
data.write.format('delta').mode('overwrite').save(path)

In [9]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forPath(spark, path)
deltaTable

In [10]:
deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = { "id": expr("id + 100") })

In [11]:
deltaTable.toDF().show()

+---+
| id|
+---+
|  7|
|108|
|  9|
|  5|
|106|
+---+



In [12]:
deltaTable.delete(condition = expr("id % 2 == 0"))

In [13]:
deltaTable.toDF().show()

+---+
| id|
+---+
|  7|
|  9|
|  5|
+---+



In [14]:
newData = spark.range(0, 20)

deltaTable.alias("oldData") \
  .merge(
    newData.alias("newData"),
    "oldData.id = newData.id") \
  .whenMatchedUpdate(set = { "id": col("newData.id") }) \
  .whenNotMatchedInsert(values = { "id": col("newData.id") }) \
  .execute()

In [15]:
deltaTable.toDF().show()

+---+
| id|
+---+
|  3|
| 12|
| 18|
|  0|
|  8|
| 11|
| 14|
| 19|
|  1|
| 16|
| 10|
| 13|
|  4|
|  7|
| 15|
|  6|
|  9|
| 17|
|  2|
|  5|
+---+



In [16]:
deltaTable.toDF().write.format('delta').mode('overwrite').save(path)

In [17]:
deltaTable = DeltaTable.forPath(spark, path)
deltaTable.toDF().show()

+---+
| id|
+---+
|  3|
| 12|
| 18|
|  0|
|  8|
| 11|
| 14|
| 19|
|  1|
| 16|
| 10|
| 13|
|  4|
|  7|
| 15|
|  6|
|  9|
| 17|
|  2|
|  5|
+---+



In [20]:
!jupyter nbconvert --to script 4.simple-delta-lake.ipynb

[NbConvertApp] Converting notebook 4.simple-delta-lake.ipynb to script
[NbConvertApp] Writing 1934 bytes to 4.simple-delta-lake.py


In [25]:
!/opt/spark/bin/spark-submit \
  --packages io.delta:delta-core_2.12:1.0.0 \
  --conf "spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension" \
  --conf "spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog" \
  --master spark://master:7077 \
  /home/data/4.simple-delta-lake.py

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-163ec01f-93b1-4c65-ba27-d896a33bd5af;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 708ms :: artifacts dl 18ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core

22/01/12 08:35:57 INFO StandaloneAppClient$ClientEndpoint: Connecting to master spark://master:7077...
22/01/12 08:35:57 INFO TransportClientFactory: Successfully created connection to master/172.20.0.4:7077 after 58 ms (0 ms spent in bootstraps)
22/01/12 08:35:57 INFO StandaloneSchedulerBackend: Connected to Spark cluster with app ID app-20220112083557-0003
22/01/12 08:35:57 INFO StandaloneAppClient$ClientEndpoint: Executor added: app-20220112083557-0003/0 on worker-20220112082535-172.20.0.5-41369 (172.20.0.5:41369) with 2 core(s)
22/01/12 08:35:57 INFO StandaloneSchedulerBackend: Granted executor ID app-20220112083557-0003/0 on hostPort 172.20.0.5:41369 with 2 core(s), 1024.0 MiB RAM
22/01/12 08:35:57 INFO Utils: Successfully started service 'org.apache.spark.network.netty.NettyBlockTransferService' on port 36549.
22/01/12 08:35:57 INFO NettyBlockTransferService: Server created on c469952c788b:36549
22/01/12 08:35:57 INFO BlockManager: Using org.apache.spark.storage.RandomBlockReplic

22/01/12 08:36:16 INFO TaskSetManager: Starting task 0.0 in stage 0.0 (TID 0) (172.20.0.5, executor 0, partition 0, PROCESS_LOCAL, 5016 bytes) taskResourceAssignments Map()
22/01/12 08:36:16 INFO TaskSetManager: Starting task 1.0 in stage 0.0 (TID 1) (172.20.0.5, executor 0, partition 1, PROCESS_LOCAL, 5317 bytes) taskResourceAssignments Map()
22/01/12 08:36:17 INFO BlockManagerInfo: Added broadcast_3_piece0 in memory on 172.20.0.5:40775 (size: 68.2 KiB, free: 366.2 MiB)
22/01/12 08:36:22 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on 172.20.0.5:40775 (size: 25.4 KiB, free: 366.2 MiB)
22/01/12 08:36:23 INFO BlockManagerInfo: Added broadcast_1_piece0 in memory on 172.20.0.5:40775 (size: 28.0 KiB, free: 366.2 MiB)
22/01/12 08:36:23 INFO BlockManagerInfo: Added broadcast_2_piece0 in memory on 172.20.0.5:40775 (size: 25.4 KiB, free: 366.2 MiB)
22/01/12 08:36:25 INFO TaskSetManager: Starting task 2.0 in stage 0.0 (TID 2) (172.20.0.5, executor 0, partition 2, PROCESS_LOCAL, 521

22/01/12 08:36:30 INFO BlockManagerInfo: Added rdd_15_14 in memory on 172.20.0.5:40775 (size: 401.0 B, free: 366.1 MiB)
22/01/12 08:36:30 INFO TaskSetManager: Starting task 16.0 in stage 1.0 (TID 17) (172.20.0.5, executor 0, partition 16, NODE_LOCAL, 4446 bytes) taskResourceAssignments Map()
22/01/12 08:36:30 INFO TaskSetManager: Finished task 14.0 in stage 1.0 (TID 15) in 291 ms on 172.20.0.5 (executor 0) (13/50)
22/01/12 08:36:30 INFO BlockManagerInfo: Added rdd_15_15 in memory on 172.20.0.5:40775 (size: 384.0 B, free: 366.1 MiB)
22/01/12 08:36:30 INFO BlockManagerInfo: Added rdd_15_16 in memory on 172.20.0.5:40775 (size: 463.0 B, free: 366.1 MiB)
22/01/12 08:36:30 INFO TaskSetManager: Starting task 17.0 in stage 1.0 (TID 18) (172.20.0.5, executor 0, partition 17, NODE_LOCAL, 4446 bytes) taskResourceAssignments Map()
22/01/12 08:36:30 INFO TaskSetManager: Finished task 15.0 in stage 1.0 (TID 16) in 322 ms on 172.20.0.5 (executor 0) (14/50)
22/01/12 08:36:30 INFO TaskSetManager: Start

22/01/12 08:36:32 INFO BlockManagerInfo: Added rdd_15_42 in memory on 172.20.0.5:40775 (size: 619.0 B, free: 366.0 MiB)
22/01/12 08:36:32 INFO BlockManagerInfo: Added rdd_15_40 in memory on 172.20.0.5:40775 (size: 357.0 B, free: 366.0 MiB)
22/01/12 08:36:32 INFO TaskSetManager: Starting task 43.0 in stage 1.0 (TID 37) (172.20.0.5, executor 0, partition 43, NODE_LOCAL, 4446 bytes) taskResourceAssignments Map()
22/01/12 08:36:32 INFO TaskSetManager: Finished task 42.0 in stage 1.0 (TID 36) in 135 ms on 172.20.0.5 (executor 0) (33/50)
22/01/12 08:36:32 INFO TaskSetManager: Starting task 44.0 in stage 1.0 (TID 38) (172.20.0.5, executor 0, partition 44, NODE_LOCAL, 4446 bytes) taskResourceAssignments Map()
22/01/12 08:36:32 INFO TaskSetManager: Finished task 40.0 in stage 1.0 (TID 35) in 188 ms on 172.20.0.5 (executor 0) (34/50)
22/01/12 08:36:32 INFO BlockManagerInfo: Added rdd_15_43 in memory on 172.20.0.5:40775 (size: 500.0 B, free: 366.0 MiB)
22/01/12 08:36:32 INFO TaskSetManager: Start

...

22/01/12 08:37:47 INFO TaskSetManager: Starting task 23.0 in stage 70.0 (TID 1829) (172.20.0.5, executor 0, partition 23, NODE_LOCAL, 4446 bytes) taskResourceAssignments Map()
22/01/12 08:37:47 INFO TaskSetManager: Finished task 21.0 in stage 70.0 (TID 1827) in 99 ms on 172.20.0.5 (executor 0) (20/50)
22/01/12 08:37:47 INFO BlockManagerInfo: Added rdd_255_22 in memory on 172.20.0.5:40775 (size: 510.0 B, free: 365.8 MiB)
22/01/12 08:37:47 INFO TaskSetManager: Finished task 22.0 in stage 70.0 (TID 1828) in 71 ms on 172.20.0.5 (executor 0) (21/50)
22/01/12 08:37:47 INFO TaskSetManager: Starting task 24.0 in stage 70.0 (TID 1830) (172.20.0.5, executor 0, partition 24, NODE_LOCAL, 4446 bytes) taskResourceAssignments Map()
22/01/12 08:37:47 INFO BlockManagerInfo: Added rdd_255_23 in memory on 172.20.0.5:40775 (size: 355.0 B, free: 365.8 MiB)
22/01/12 08:37:47 INFO BlockManagerInfo: Added rdd_255_24 in memory on 172.20.0.5:40775 (size: 602.0 B, free: 365.8 MiB)
22/01/12 08:37:47 INFO TaskSetM

22/01/12 08:37:48 INFO BlockManagerInfo: Added rdd_255_44 in memory on 172.20.0.5:40775 (size: 291.0 B, free: 365.8 MiB)
22/01/12 08:37:48 INFO BlockManagerInfo: Added rdd_255_45 in memory on 172.20.0.5:40775 (size: 457.0 B, free: 365.8 MiB)
22/01/12 08:37:48 INFO TaskSetManager: Starting task 46.0 in stage 70.0 (TID 1849) (172.20.0.5, executor 0, partition 46, NODE_LOCAL, 4446 bytes) taskResourceAssignments Map()
22/01/12 08:37:48 INFO TaskSetManager: Finished task 45.0 in stage 70.0 (TID 1848) in 75 ms on 172.20.0.5 (executor 0) (40/50)
22/01/12 08:37:48 INFO TaskSetManager: Starting task 47.0 in stage 70.0 (TID 1850) (172.20.0.5, executor 0, partition 47, NODE_LOCAL, 4446 bytes) taskResourceAssignments Map()
22/01/12 08:37:48 INFO TaskSetManager: Finished task 44.0 in stage 70.0 (TID 1847) in 119 ms on 172.20.0.5 (executor 0) (41/50)
22/01/12 08:37:48 INFO BlockManagerInfo: Added rdd_255_46 in memory on 172.20.0.5:40775 (size: 360.0 B, free: 365.8 MiB)
22/01/12 08:37:48 INFO TaskSet

22/01/12 08:37:49 INFO FileSourceStrategy: Pushed Filters: 
22/01/12 08:37:49 INFO FileSourceStrategy: Post-Scan Filters: 
22/01/12 08:37:49 INFO FileSourceStrategy: Output Data Schema: struct<id: bigint>
22/01/12 08:37:49 INFO MemoryStore: Block broadcast_92 stored as values in memory (estimated size 299.0 KiB, free 360.5 MiB)
22/01/12 08:37:49 INFO MemoryStore: Block broadcast_92_piece0 stored as bytes in memory (estimated size 26.0 KiB, free 360.5 MiB)
22/01/12 08:37:49 INFO BlockManagerInfo: Added broadcast_92_piece0 in memory on c469952c788b:36549 (size: 26.0 KiB, free: 365.7 MiB)
22/01/12 08:37:49 INFO SparkContext: Created broadcast 92 from showString at NativeMethodAccessorImpl.java:0
22/01/12 08:37:49 INFO Snapshot: [tableId=241f05d7-a0e1-429c-b470-4e0fe4ebbe2d] DELTA: Filtering files for query
22/01/12 08:37:49 INFO BlockManagerInfo: Removed broadcast_72_piece0 on 172.20.0.5:40775 in memory (size: 28.0 KiB, free: 365.7 MiB)
22/01/12 08:37:49 INFO BlockManagerInfo: Removed bro

22/01/12 08:37:49 INFO TaskSetManager: Starting task 18.0 in stage 73.0 (TID 1877) (172.20.0.5, executor 0, partition 18, PROCESS_LOCAL, 4457 bytes) taskResourceAssignments Map()
22/01/12 08:37:49 INFO TaskSetManager: Finished task 16.0 in stage 73.0 (TID 1875) in 20 ms on 172.20.0.5 (executor 0) (17/50)
22/01/12 08:37:49 INFO TaskSetManager: Starting task 19.0 in stage 73.0 (TID 1878) (172.20.0.5, executor 0, partition 19, PROCESS_LOCAL, 4457 bytes) taskResourceAssignments Map()
22/01/12 08:37:49 INFO TaskSetManager: Finished task 17.0 in stage 73.0 (TID 1876) in 23 ms on 172.20.0.5 (executor 0) (18/50)
22/01/12 08:37:49 INFO TaskSetManager: Starting task 20.0 in stage 73.0 (TID 1879) (172.20.0.5, executor 0, partition 20, PROCESS_LOCAL, 4457 bytes) taskResourceAssignments Map()
22/01/12 08:37:49 INFO TaskSetManager: Finished task 18.0 in stage 73.0 (TID 1877) in 16 ms on 172.20.0.5 (executor 0) (19/50)
22/01/12 08:37:49 INFO TaskSetManager: Starting task 21.0 in stage 73.0 (TID 1880)

22/01/12 08:37:49 INFO TaskSetManager: Starting task 47.0 in stage 73.0 (TID 1906) (172.20.0.5, executor 0, partition 47, PROCESS_LOCAL, 4457 bytes) taskResourceAssignments Map()
22/01/12 08:37:49 INFO TaskSetManager: Finished task 45.0 in stage 73.0 (TID 1904) in 16 ms on 172.20.0.5 (executor 0) (46/50)
22/01/12 08:37:50 INFO TaskSetManager: Starting task 48.0 in stage 73.0 (TID 1907) (172.20.0.5, executor 0, partition 48, PROCESS_LOCAL, 4457 bytes) taskResourceAssignments Map()
22/01/12 08:37:50 INFO TaskSetManager: Finished task 46.0 in stage 73.0 (TID 1905) in 19 ms on 172.20.0.5 (executor 0) (47/50)
22/01/12 08:37:50 INFO TaskSetManager: Starting task 49.0 in stage 73.0 (TID 1908) (172.20.0.5, executor 0, partition 49, PROCESS_LOCAL, 4457 bytes) taskResourceAssignments Map()
22/01/12 08:37:50 INFO TaskSetManager: Finished task 47.0 in stage 73.0 (TID 1906) in 26 ms on 172.20.0.5 (executor 0) (48/50)
22/01/12 08:37:50 INFO TaskSetManager: Finished task 48.0 in stage 73.0 (TID 1907)